# AccuWeather Client

## Introduction

This notebook objective is to incrementely and interactively test and develop accuweather API in order to build the core logics of the _Meteo French API_

## Code

### Setup

In [24]:
# Imports
import os
import requests
from dotenv import dotenv_values


SCRIPT_DIR = os.path.abspath('')

# Constants
AW_BASE_URL = "http://dataservice.accuweather.com"
GEOPOSITION_URL = AW_BASE_URL + "/locations/v1/cities/geoposition/search"
CONDITIONS_URL = AW_BASE_URL + "/currentconditions/v1"

# Parameters
ENV_FILE = os.path.join(SCRIPT_DIR, "../.env.dev")
SECRET_FILE = os.path.join(SCRIPT_DIR, "../.env.secret")

## For test
TOKEN_TEST_URL = AW_BASE_URL + "/locations/v1/regions"
LATLNG_PARIS = (48.870502, 2.304897)

# Env variables
config = {
    **dotenv_values(ENV_FILE),  # load shared development variables
    **dotenv_values(SECRET_FILE),  # load sensitive variables
}
ACCUWEATHER_TOKEN = config['ACCUWEATHER_TOKEN']


### Quick Tests

In [25]:
# Test accuweather token
params = {"apikey": ACCUWEATHER_TOKEN}
url = TOKEN_TEST_URL
res = requests.get(url, params=params)
print(res)

if not res.ok:
    print("ERROR ({})".format(res.status_code))
else:
    print("OK")

<Response [200]>
OK


In [26]:
# Get locationKey
url = GEOPOSITION_URL
latlng = LATLNG_PARIS
params = {
    "apikey": ACCUWEATHER_TOKEN,
    "q": "{},{}".format(*latlng),
}

res = requests.get(url, params=params)
locationKey = None
if res.ok:
    locationKey = res.json().get('Key')
else:
    print("ERROR ({})".format(res.status_code))
print(locationKey)

2608429


In [53]:
# Utils functions
def homogenize_unit(value_obj):
    # we will homogeneize in m
    value_obj_s = value_obj["Metric"]
    # mm
    if value_obj_s["UnitType"] == 3:
        return value_obj_s["Value"] * 0.001
    # cm
    if value_obj_s["UnitType"] == 4:
        return value_obj_s["Value"] * 0.01
    # m
    if value_obj_s["UnitType"] == 5:
        return value_obj_s["Value"]
    # km
    if value_obj_s["UnitType"] == 6:
        return value_obj_s["Value"] * 1000


# Get visibility
url = CONDITIONS_URL + "/{}".format(locationKey)
params = {
    "apikey": ACCUWEATHER_TOKEN,
    "details": True,
}

res = requests.get(url, params=params)
visibility = None
precipitation = None
if res.ok:
    data = res.json()
    if len(data):
        visibility_obj = data[0].get('Visibility')
        precipitation_obj = data[0].get('Precip1hr')
        print(precipitation_obj)
        visibility = homogenize_unit(visibility_obj)
        precipitation = homogenize_unit(precipitation_obj)
else:
    print("ERROR ({})".format(res.status_code))
return_data = {
    "visibility": visibility,
    "precipitation": precipitation
}
print(return_data)

{'Metric': {'Value': 0.0, 'Unit': 'mm', 'UnitType': 3}, 'Imperial': {'Value': 0.0, 'Unit': 'in', 'UnitType': 1}}
{'visibility': 9700.0, 'precipitation': 0.0}


In [56]:
# Get uv index
url = CONDITIONS_URL + "/{}".format(locationKey)
params = {
    "apikey": ACCUWEATHER_TOKEN,
    "details": True,
}

res = requests.get(url, params=params)
uv_index = None
if res.ok:
    data = res.json()
    if len(data):
        uv_index = data[0].get('UVIndex')
else:
    print("ERROR ({})".format(res.status_code))

print(uv_index)

0
